In [ ]:
pip install word2number

  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5580 sha256=5cb544e25ab6d7e8693d4383df6f8ed9d97139c166e7c6fe25e3d6b79f993594
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from word2number import w2n
import pandas as pd 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [ ]:
def text2int (textnum, numwords={}):
    if not numwords:
        units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
        ]
        tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]
        scales = ["hundred", "thousand", "million", "billion", "trillion"]
        numwords["and"] = (1, 0)
        for idx, word in enumerate(units):  numwords[word] = (1, idx)
        for idx, word in enumerate(tens):       numwords[word] = (1, idx * 10)
        for idx, word in enumerate(scales): numwords[word] = (10 ** (idx * 3 or 2), 0)
    ordinal_words = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
    ordinal_endings = [('ieth', 'y'), ('th', '')]
    textnum = textnum.replace('-', ' ')
    current = result = 0
    curstring = ""
    onnumber = False
    for word in textnum.split():
        if word in ordinal_words:
            scale, increment = (1, ordinal_words[word])
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0
            onnumber = True
        else:
            for ending, replacement in ordinal_endings:
                if word.endswith(ending):
                    word = "%s%s" % (word[:-len(ending)], replacement)
            if word not in numwords:
                if onnumber:
                    curstring += repr(result + current) + " "
                curstring += word + " "
                result = current = 0
                onnumber = False
            else:
                scale, increment = numwords[word]
                current = current * scale + increment
                if scale > 100:
                    result += current
                    current = 0
                onnumber = True
    if onnumber:
        curstring += repr(result + current)
    return curstring

In [ ]:
#@title USER INPUT
input = "i need 7 chairs, three tables , 17 steelplates" #@param {type:"string"}
input = input.lower()
input = text2int(input)

In [ ]:
sent = preprocess(input)

In [ ]:
sent

[('i', 'NNS'),
 ('need', 'VBP'),
 ('7', 'CD'),
 ('chairs', 'NNS'),
 (',', ','),
 ('3', 'CD'),
 ('tables', 'NNS'),
 (',', ','),
 ('17', 'CD'),
 ('steelplates', 'NNS')]

In [ ]:
list = []
i = 0
for num_1 in range(0,len(sent)-1):
  tup_1 = sent[num_1]
  t1 = tup_1[1]
  i = i+1
  tup_2 = sent[i]
  t2 = tup_2[1]
  if t1 == "CD" and t2 == "NNS":
    list.append((tup_1[0],tup_2[0]))
  if t1 == "DT" and t2 == "NN":
    list.append((tup_1[0],tup_2[0]))

In [ ]:
list

[('7', 'chairs'), ('3', 'tables'), ('17', 'steelplates')]

In [ ]:
A1 = [lis[-1] for lis in list]
A1

['chairs', 'tables', 'steelplates']

In [ ]:
A2 = [lis[0] for lis in list]
A2 = A2

In [ ]:
for li in range(0, len(A2)):
  if A2[li] == "a":
    A2[li] = "1"
  if A2[li] == "an":
    A2[li] = "1"     

In [ ]:
A2

['7', '3', '17']

In [ ]:
print(A1)
print(A2)

['chairs', 'tables', 'steelplates']
['7', '3', '17']


In [ ]:
final_frame = pd.DataFrame({'Object' : A1,
                                'Count' : A2 }, 
                                columns=['Object','Count'])

In [ ]:
final_frame

,Object,Count
0,chairs,7
1,tables,3
2,steelplates,17


In [ ]:
ob_csv = final_frame.to_csv ('file1.csv')